In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

rootpath = '/media/share/data/kaggle/ieee-camera/'

# Preprocess

In [4]:
def df_generater(rootpath, dataset):
    # find all the camera
    subdir = os.path.join(rootpath, dataset)
    n_class = os.listdir(subdir)
    df_temp = []

    for i in n_class:
        for fname in os.listdir(os.path.join(subdir, i)):
            df_temp.append((i, fname))

    df = pd.DataFrame(df_temp, columns=['class', 'fname'])
    print('class:', n_class)
    print('number of file:', df.shape[0])
    
    return df, n_class


def preprocess(rootpath, dataset):
    # read image file and check channels    
    df, _ = df_generater(rootpath, dataset)
    subpath = os.path.join(rootpath, dataset)

    img_para = []
    for idx in range(len(df)):
        img = plt.imread(os.path.join(subpath, df['class'][idx]) + '/' + df['fname'][idx])
        if img.shape[0] > img.shape[1]:
            img = img.transpose(1, 0, 2)
        # image size as a feature
        img_para.append((img.shape[1], img.shape[0]/img.shape[1]))
    
    img_para = pd.DataFrame(img_para, columns=['size', 'ratio'])
    df = pd.concat([df, img_para], 1)
    
    return df


def train_val_split(df, train_size=0.8):
    from sklearn.preprocessing import LabelEncoder
    n_class = ['Motorola-X', 
               'HTC-1-M7', 
               'iPhone-4s', 
               'Samsung-Galaxy-Note3', 
               'Motorola-Nexus-6', 
               'LG-Nexus-5x', 
               'Samsung-Galaxy-S4', 
               'Motorola-Droid-Maxx', 
               'iPhone-6', 
               'Sony-NEX-7']
    le = LabelEncoder().fit(n_class)
    
    tidx = np.arange(len(df))
    np.random.shuffle(tidx)
    tidx = pd.DataFrame(tidx, columns=['idx'])
    df_0 = pd.concat([df, tidx], 1)
    df_0 = df_0.sort_values(by=['idx'])
    
    trains = int(train_size * len(df))
    X_train = df['image'][:trains] / 255.
    X_val = df['image'][trains:] / 255.
    y_train = le.transform(df['class'][:trains])
    y_val = le.transform(df['class'][trains:])
    
    return X_train, X_val, y_train, y_val


def image_crop(rootpath, dataset='train', crop_size=256):
    from skimage.util import crop
    df, _ = df_generater(rootpath, dataset)
    subpath = os.path.join(rootpath, dataset)
    
    img_file = []
    label = []
    for idx in range(len(df)):
        img = plt.imread(os.path.join(subpath, df['class'][idx]) + '/' + df['fname'][idx])
        if img.shape[0] > img.shape[1]:
            img = img.transpose(1, 0, 2)

        h1, w1, _ = img.shape
        h2 = int(h1/int(h1/crop_size))
        w2 = int(w1/int(w1/crop_size))
        
        for i in range(int(h1/crop_size)):
            hr = np.random.randint(i*h2, (i+1)*h2-crop_size)
            for j in range(int(w1/crop_size)):
                wr = np.random.randint(j*w2, (j+1)*w2-crop_size)
                img_file.append(img[hr:hr+crop_size, wr:wr+crop_size] / 255.)
                label.append(df['class'][idx])

    img_array = pd.DataFrame(img_file, columns=['class', 'image'])
    
    return img_array

In [8]:
from sklearn.model_selection import StratifiedKFold
df_train, _ = df_generater(rootpath, dataset='train')
n_fold = 5
skf = StratifiedKFold(n_fold, shuffle=True, random_state=np.random)
for train_idx, val_idx in skf.split(df_train['fname'], df_train['class']):
    break

class: ['Motorola-X', 'HTC-1-M7', 'iPhone-4s', 'Samsung-Galaxy-Note3', 'Motorola-Nexus-6', 'LG-Nexus-5x', 'Samsung-Galaxy-S4', 'Motorola-Droid-Maxx', 'iPhone-6', 'Sony-NEX-7']
number of file: 2750


# CNN model

In [ ]:
import keras.backend as K
from keras.applications import inception_resnet_v2
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Conv2D, Concatenate, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau
import datetime


K.clear_session()

base_model = inception_resnet_v2.InceptionResNetV2(include_top=False, weights='imagenet', input_shape=(256, 256, 3))

x = base_model.output
x = Dense(1024, activation='relu')(x)
prediction = Dense(9, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=prediction)
model.summary()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Trainning

In [ ]:
X_train, X_val, y_train, y_val = image_crop(rootpath, 'train', crop_size=256)

In [ ]:
train_gen = ImageDataGenerator(featurewise_std_normalization=True,
                               samplewise_std_normalization=True, 
                               rotation_range=45, 
                               horizontal_flip=True, 
                               vertical_flip=True)

train_gen.fit(X_train)

In [ ]:
model_checkpoint = ModelCheckpoint('ieeev2-{epoch:02d}-{val_loss:.5f}.hdf5',
                                   monitor='val_loss', save_best_only=True, save_weights_only=True, )

model_earlystop = EarlyStopping(patience=64, monitor='val_loss')
adlr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=16, verbose=0, mode='auto', epsilon=0.0001, cooldown=0, min_lr=1e-6)

batch_size = 128

train_history = model.fit_generator(train_gen.flow(X_train, y_train, batch_size), 
                                    epochs=2**1, steps_per_epoch=(len(X_train)//batch_size), 
                                    validation_data=train_gen.flow(X_val, y_val, batch_size), 
                                    validation_steps=(len(X_val)//batch_size),
                                    verbose=0, callbacks=[model_checkpoint, adlr])

In [ ]:
testing = os.path